# Delta Airlines Review

Read data from airline reviews and service ratings datasets.

In [88]:
import pandas as pd
import numpy as np

# Load the datasets
service_ratings = pd.read_csv('data/raw_data/Delta-Service-RatingbyServiceType.csv')
airline_reviews = pd.read_csv('data/raw_data/Delta_Airline_Review_Dataset-Asof02172023.csv')

airline_reviews.head()


,Customer_ID,star rating,date,Seat Type,routes,traveler_types,country,reviews
0,1,4,17th February 2023,Economy Class,New York to Tel Aviv,Solo Leisure,United States,"✅ Trip Verified | First, travel starts 6pm ar..."
1,2,1,16th February 2023,Economy Class,Milwaukee to Ft Lauderdale,Business,United States,✅ Trip Verified | Delta did not issue any wea...
2,3,1,10th February 2023,Economy Class,New York to Charlotte,Business,United States,"✅ Trip Verified | Per our pilot, there would ..."
3,4,1,8th February 2023,Premium Economy,Atlanta to Cape Town,Family Leisure,United States,✅ Trip Verified | We flew from Atlanta to Cap...
4,5,2,6th February 2023,Economy Class,Dallas to Atlanta,Solo Leisure,United Kingdom,✅ Trip Verified | Checkin staff always seem u...


### Data Preprocessing

Split the reviews column into trip_verified and content columns. 

In [89]:
# Initialize columns with empty strings
airline_reviews['trip_verified'] = ''
airline_reviews['content'] = ''

# Split the 'reviews' column using the pipe separator '|', if present
split_data = airline_reviews['reviews'].str.split('|', n=1, expand=True)

airline_reviews.loc[split_data[1].notna(), 'trip_verified'] = split_data[0].str.strip()
airline_reviews.loc[split_data[1].notna(), 'content'] = split_data[1].str.strip()
airline_reviews.loc[split_data[1].isna(), 'content'] = airline_reviews['reviews']

# Drop the original 'reviews' column
airline_reviews.drop(columns=['reviews'], inplace=True)

In [90]:
airline_reviews['trip_verified'] = airline_reviews['trip_verified'].apply(lambda x: 'Verified' if x == '✅ Trip Verified' else 'Not Verified' if x in ['Not Verified', 'Unverified'] else '')

airline_reviews.head()

,Customer_ID,star rating,date,Seat Type,routes,traveler_types,country,trip_verified,content
0,1,4,17th February 2023,Economy Class,New York to Tel Aviv,Solo Leisure,United States,Verified,"First, travel starts 6pm arrival at the airpor..."
1,2,1,16th February 2023,Economy Class,Milwaukee to Ft Lauderdale,Business,United States,Verified,Delta did not issue any weather waivers so cus...
2,3,1,10th February 2023,Economy Class,New York to Charlotte,Business,United States,Verified,"Per our pilot, there would be a slight delay d..."
3,4,1,8th February 2023,Premium Economy,Atlanta to Cape Town,Family Leisure,United States,Verified,We flew from Atlanta to Cape Town and Back aft...
4,5,2,6th February 2023,Economy Class,Dallas to Atlanta,Solo Leisure,United Kingdom,Verified,"Checkin staff always seem unfriendly, no matte..."


Get the list of services and their ratings an merge the datasets

In [94]:
# Prepare the columns for the ratings
rating_columns = [
    'Seat Comfort', 'Cabin Staff Service', 'Ground Service', 'Value For Money',
    'Food & Beverages', 'Inflight Entertainment', 'Wifi & Connectivity'
]

# Initialize an empty DataFrame to store ratings for each review
ratings_list = []
current_review = {}

# Iterate through the service ratings
for index, row in service_ratings.iterrows():
    attribute = row['attributes']
    rating = row['ratings']
    current_review[attribute] = rating
    
    # Check if the next row starts a new set of ratings or if we are at the end
    if (index + 1 < len(service_ratings) and service_ratings.loc[index + 1, 'attributes'] == 'Seat Comfort') or (index == len(service_ratings) - 1):
        # Ensure all attributes are present, fill missing with NaN
        for column in rating_columns:
            if column not in current_review:
                current_review[column] = np.nan
        ratings_list.append(current_review)
        current_review = {}

ratings_df = pd.DataFrame(ratings_list)

ratings_df.head()

,Seat Comfort,Cabin Staff Service,Ground Service,Value For Money,Food & Beverages,Inflight Entertainment,Wifi & Connectivity
0,2,3.0,1.0,3,NaN,NaN,NaN
1,3,5.0,3.0,1,3.0,3.0,2.0
2,1,1.0,1.0,1,1.0,1.0,1.0
3,2,3.0,1.0,1,3.0,3.0,3.0
4,3,4.0,1.0,1,2.0,1.0,1.0


In [95]:
# Check the lengths of both DataFrames
len_airline_reviews = len(airline_reviews)
len_ratings_df = len(ratings_df)

print(f"Number of reviews: {len_airline_reviews}")
print(f"Number of ratings sets: {len_ratings_df}")

# Align the lengths of both DataFrames
if len_airline_reviews > len_ratings_df:
    print("There are more reviews than ratings sets.")
    additional_rows = len_airline_reviews - len_ratings_df
    additional_ratings_df = pd.DataFrame(np.nan, index=range(additional_rows), columns=ratings_df.columns)
    ratings_df = pd.concat([ratings_df, additional_ratings_df], ignore_index=True)
elif len_airline_reviews < len_ratings_df:
    print("There are more ratings sets than reviews.")
    ratings_df = ratings_df.iloc[:len_airline_reviews]
else:
    print("The number of reviews and ratings sets match.")


Number of reviews: 2689
Number of ratings sets: 2589
There are more reviews than ratings sets.


In [96]:
# Merge the DataFrames
merged_reviews = pd.concat([airline_reviews.reset_index(drop=True), ratings_df.reset_index(drop=True)], axis=1)

merged_reviews.head()

,Customer_ID,star rating,date,Seat Type,routes,traveler_types,country,trip_verified,content,Seat Comfort,Cabin Staff Service,Ground Service,Value For Money,Food & Beverages,Inflight Entertainment,Wifi & Connectivity
0,1,4,17th February 2023,Economy Class,New York to Tel Aviv,Solo Leisure,United States,Verified,"First, travel starts 6pm arrival at the airpor...",2.0,3.0,1.0,3.0,NaN,NaN,NaN
1,2,1,16th February 2023,Economy Class,Milwaukee to Ft Lauderdale,Business,United States,Verified,Delta did not issue any weather waivers so cus...,3.0,5.0,3.0,1.0,3.0,3.0,2.0
2,3,1,10th February 2023,Economy Class,New York to Charlotte,Business,United States,Verified,"Per our pilot, there would be a slight delay d...",1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,4,1,8th February 2023,Premium Economy,Atlanta to Cape Town,Family Leisure,United States,Verified,We flew from Atlanta to Cape Town and Back aft...,2.0,3.0,1.0,1.0,3.0,3.0,3.0
4,5,2,6th February 2023,Economy Class,Dallas to Atlanta,Solo Leisure,United Kingdom,Verified,"Checkin staff always seem unfriendly, no matte...",3.0,4.0,1.0,1.0,2.0,1.0,1.0


In [98]:
print("Missing percentage:")
print(merged_reviews.isnull().mean()*100)

Missing percentage:
Customer_ID                0.000000
star rating                0.000000
date                       0.000000
 Seat Type                 0.000000
routes                    15.842321
traveler_types            15.135738
country                    0.000000
trip_verified              0.000000
content                    0.000000
Seat Comfort               3.718855
Cabin Staff Service        3.941986
Ground Service            19.003347
Value For Money            3.718855
Food & Beverages          22.164373
Inflight Entertainment    29.007066
Wifi & Connectivity       54.555597
dtype: float64


Handle missing values

In [99]:
# Fill missing values in rating columns with 0
merged_reviews[rating_columns] = merged_reviews[rating_columns].fillna(0)

# Fill missing values in 'trip_verified' and 'content' columns
merged_reviews['trip_verified'].fillna('Not Verified', inplace=True)
merged_reviews['content'].fillna('N/A', inplace=True)

/var/folders/js/8x4jrcf12dqd5ms5f18629cc0000gn/T/ipykernel_36299/862703763.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_reviews['trip_verified'].fillna('Not Verified', inplace=True)
/var/folders/js/8x4jrcf12dqd5ms5f18629cc0000gn/T/ipykernel_36299/862703763.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are s

In [100]:
print("Missing percentage:")
print(merged_reviews.isnull().mean()*100)

Missing percentage:
Customer_ID                0.000000
star rating                0.000000
date                       0.000000
 Seat Type                 0.000000
routes                    15.842321
traveler_types            15.135738
country                    0.000000
trip_verified              0.000000
content                    0.000000
Seat Comfort               0.000000
Cabin Staff Service        0.000000
Ground Service             0.000000
Value For Money            0.000000
Food & Beverages           0.000000
Inflight Entertainment     0.000000
Wifi & Connectivity        0.000000
dtype: float64


In [101]:
merged_reviews.dropna(subset=['routes', 'traveler_types'], inplace=True)

In [102]:
print("Missing percentage:")
print(merged_reviews.isnull().mean()*100)

Missing percentage:
Customer_ID               0.0
star rating               0.0
date                      0.0
 Seat Type                0.0
routes                    0.0
traveler_types            0.0
country                   0.0
trip_verified             0.0
content                   0.0
Seat Comfort              0.0
Cabin Staff Service       0.0
Ground Service            0.0
Value For Money           0.0
Food & Beverages          0.0
Inflight Entertainment    0.0
Wifi & Connectivity       0.0
dtype: float64


Save the final dataset for Tableau visualization

In [104]:
# Save the merged dataset with ratings as new columns
merged_filename = 'data/merged_data/Merged_Delta_Airline_Reviews.csv'
merged_reviews.to_csv(merged_filename, index=False)